# Hash Table

**Prerequisites**: Accumulator

A hash table is both a data structure and a strategy for quickly locating an item in a sequence of key-value pairs.

A hash table associates keys to values. Each key in a hash table is unique.

## Hash function

A hash table needs a hash function that returns the same value for each unique key.

### Exercise

Complete the code below for the function `hash(key: str, m: int) -> int` which returns a hash location for the given key.

Note that this will override Python's built-in `hash()` function. This is intentional.

For this exercise, we will use the [rolling polynomial hash algorithm](https://cp-algorithms.com/string/string-hashing.html). Use the following values:

- `m = <size of hash table>`
- `p = 127`

In [ ]:
def hash(key: str, size: int) -> int:
    """Calculates the hash of a string using rolling polynomial algorithm."""
    p = 127
    total = 0
    # Write code below to calculate the hash location (integer) from the key.
    

## Hash table using array

Let's start with a hash table of size 10. This means the hash table starts out with 10 locations for storing key-value pairs, though at the time of initialisation none of them will be in use. (In practice you can start out with a hash table of any size.)

How do we know if a slot is in use? We need a **sentinel value** (see Lesson 6); for this exercise, we will use Python's `None` value as our sentinel.

A slot will contain:
- `None` if it is not in use
- `(key, value)` stored as a 2-element `tuple` if it is in use

In [ ]:
SIZE = 10

table = [None] * SIZE
table

I'll provide two helper functions for inspecting the data in the table:

- `clear(table: list) -> None`  
  Remove all entries from the hash table

- `inspect(table: list) -> None`  
  Display the contents of the hash table for inspection
  

### Exercise

Now we need some functions for manipulating this table. Let's write some code to define the following functions:

- `add(table: list, key: str, value: Any) -> None`  
  Adds a key-value pair to the hash table

- `get(table: list, key: str) -> Any`  
  Return the value associated with the key from the table

- `remove(table: list, key: str) -> None`  
  Remove a key-value pair from the table

In [ ]:
from typing import Any


# Helper functions
def clear(table: list) -> None:
    """Remove all entries from the table"""
    # Use this function for resetting your table
    for i in range(len(table)):
        table[i] = None

        
def inspect(table: list) -> None:
    """Display the contents of the hash table for inspection"""
    for i in range(len(table)):
        if table[i]:  # ignore empty slots
            key, value = table[i]
            print(f"[slot {i}] '{key}': {value}")


# Manipulation functions
def add(table: list, key: str, value: Any) -> None:
    """Adds a key-value pair to the hash table"""
    pass


def get(table: list, key: str) -> Any:
    """Return the value associated with the key from the table"""
    pass


def remove(table: list, key: str) -> None:
    """Remove a key-value pair from the table"""
    index = hash(key, SIZE)
    pass

    
clear(table)
add(table, 'Apricot', 0)
add(table, 'Banana', 1)
print(get(table, 'Apricot'))
print(get(table, 'Banana'))
inspect(table)

# Expected output:
# 0
# 1
# [slot 0] 'Apricot': 0
# [slot 5] 'Banana': 1

## Key collisions

This is all well and good, but sometimes two keys can hash to the same location:

In [ ]:
clear(table)
add(table, 'Apricot', 0)
add(table, 'Elderberry', 1)
print(get(table, 'Apricot'))
inspect(table)

Oops, `'Elderberry'` just overwrote `Apricot` ... It's easy to see why. Let's check the hash of both values:

In [ ]:
print(hash('Apricot', SIZE))
print(hash('Elderberry', SIZE))

Both keys hash to the same location, `0`. This is difficult to avoid with small hash tables, but nonetheless we can't accept this result. We need a way to handle key collisions.

## Collision resolution strategy: open addressing
*(Implementation is not in syllabus)*

One strategy is to **rehash** the key, generating another slot that we can check for availability.

There are a variety of rehashing strategies, the simplest rehashing algorithm is to increment the existing slot by 1.

We can't keep re-hashing, that will make our hash table slow. Let's cap the number of re-hashes to 5. (You can choose any number of retries in your own code.)

In [ ]:
def rehash(slot: int, size: int) -> int:
    """Rehash a slot to find another available one"""
    return (slot + 1) % size


We will need to re-implement our manipulation functions to rehash the slot until we find an available one. Starting with `add()`:

In [ ]:
def add(table: list, key: str, value: Any) -> None:
    """Adds a key-value pair to the hash table"""
    index = hash(key, SIZE)
    tries = 5
    while tries > 0:
        # If slot is empty, store key-value
        if table[index] is None:
            table[index] = (key, value)
            return
        # If slot already contains same key, update value
        table_key, table_value = table[index]
        if key == table_key:
            table[index] = (key, value)
            return
        # If slot contains different key, re-hash
        index = rehash(index, SIZE)
        tries -= 1
    # If this point is reached, no slot can be found after retrying.
    # Another strategy is required, e.g. expanding the hash table


clear(table)
add(table, 'Apricot', 0)
add(table, 'Elderberry', 1)
print(get(table, 'Apricot'))
inspect(table)

Now we also need to retrieve the value using `get()`, which also needs to be rewritten to handle rehashing.

In [ ]:
def get(table: list, key: str) -> Any:
    """Return the value associated with the key from the table"""
    index = hash(key, SIZE)
    tries = 5
    while tries > 0:
        # If slot is empty, raise KeyError
        if table[index] is None:
            raise KeyError
        # If slot already contains same key, return value
        table_key, table_value = table[index]
        if key == table_key:
            return table_value
        # If slot contains different key, re-hash
        index = rehash(index, SIZE)
        tries -= 1
    # If this point is reached, it is assumed the key is not in the table.
    raise KeyError

That was somewhat simpler than adding, but removing will be more complex. That's because removing a slot might disrupt the rehashing process - the current slot might now result in `get()` returning `None` instead of rehashing the next slot.

So we have to check subsequent slots, moving them up if necessary.

In [ ]:
def remove(table: list, key: str) -> None:
    """Remove a key-value pair from the table"""
    index = hash(key, SIZE)
    tries = 5
    found = False
    while tries > 0:
        # If slot is empty, raise KeyError
        if table[index] is None:
            raise KeyError
        # If slot already contains same key, remove key-value
        table_key, table_value = table[index]
        if key == table_key:
            table[index] = None
            found = True
            break
        # If slot contains different key, re-hash
        index = rehash(index, SIZE)
        tries -= 1
    if not found:
        raise KeyError
    
    # Check if subsequent slots need to be re-added
    index = rehash(index, SIZE)
    # - if the slot is empty, stop
    while table[index] is not None:
        table_key, table_value = table[index]
        # - if the key's hashed location does not match the slot index,
        #   remove and re-add, then check next slot.
        if hash(table_key, SIZE) != index:
            table[index] = None
            add(table, table_key, table_value)
        # - if the key's hashed location matches the slot index, check the next slot
        index = rehash(index, SIZE)

Let's check the results.

In [ ]:
clear(table)
add(table, 'Apricot', 0)
add(table, 'Banana', 1)
add(table, 'Coconut', 2)
add(table, 'Durian', 3)
add(table, 'Elderberry', 4)
print(get(table, 'Apricot'))
print(get(table, 'Banana'))
print(get(table, 'Elderberry'))
print("Initial:")
inspect(table)

remove(table, 'Apricot')
print("After removing 'Apricot':")
inspect(table)

remove(table, 'Banana')
print("After removing 'Banana':")
inspect(table)

Phew! That was a lot. Fortunately we don't need to do this for the exam practical.

## Collision resolution strategy: closed addressing
*(Implementation is not in syllabus)*

Another strategy is to use a linked list for each slot. No rehashing is needed, but we have to walk the linked list until we find a matching key.

Let's make a Linked List:

In [ ]:
class LinkedList:
    def __init__(self) -> None:
        self.head = None
    
    def __repr__(self) -> str:
        return "LinkedList()"
    
    def __str__(self) -> str:
        contents = []
        node = self.head
        while node is not None:
            contents.append(str(node))
            node = node.next()
        return '(' + ", ".join(contents) + ')'
        
    def isempty(self) -> bool:
        return self.head is None
    
    def add(self, key: str, value: Any) -> None:
        if self.head is None:
            self.head = Node(key, value)
            return
        node = self.head
        while node.next() is not None:
            node = node.next()
        node.link(Node(key, value))
    
    def get(self, key: str) -> Any:
        node = self.head
        while node is not None:
            if node.key == key:
                return node.value
            node = node.next()
        raise KeyError
    
    def remove(self, key: str) -> None:
        if self.head is None:
            raise KeyError
        if self.head.key == key:
            self.head = self.head.next()
            return
        prev, node = self.head, self.head.next()
        while node is not None:
            if node.key == key:
                prev.link(node.next())
                return
            prev, node = node, node.next()


class Node:
    def __init__(self, key: str, value: Any) -> None:
        self.key = key
        self.value = value
        self._next = None
    
    def __repr__(self) -> str:
        return f"Node('{self.key}', {self.value})"
    
    def __str__(self) -> str:
        return f"'{self.key}': {self.value}"
    
    def next(self) -> "Node":
        return self._next
    
    def link(self, node: "Node") -> None:
        """Link self to the given node"""
        self._next = node

We will initialise the table using our Linked List.

Since the structure of our table is different, we will need new `clear()` and `inspect()` helpers as well.

In [ ]:
def clear(table: list) -> None:
    """Remove all entries from the table"""
    # Use this function for resetting your table
    for i in range(len(table)):
        table[i] = LinkedList()

def inspect(table: list) -> None:
    """Display the contents of the hash table for inspection"""
    for i in range(len(table)):
        if not table[i].isempty():  # ignore empty slots
            print(f"[slot {i}] {str(table[i])}")


table = [LinkedList() for i in range(SIZE)]

And we re-implement our manipulating functions to use our LinkedList's methods:

In [ ]:
def add(table: list, key: str, value: Any) -> None:
    """Adds a key-value pair to the hash table"""
    index = hash(key, SIZE)
    table[index].add(key, value)


In [ ]:
def get(table: list, key: str) -> Any:
    """Return the value associated with the key from the table"""
    index = hash(key, SIZE)
    return table[index].get(key)


In [ ]:
def remove(table: list, key: str) -> None:
    """Remove a key-value pair from the table"""
    index = hash(key, SIZE)
    table[index].remove(key)


Let's check the results.

In [ ]:
clear(table)
add(table, 'Apricot', 0)
add(table, 'Banana', 1)
add(table, 'Coconut', 2)
add(table, 'Durian', 3)
add(table, 'Elderberry', 4)
print(get(table, 'Apricot'))
print(get(table, 'Banana'))
print(get(table, 'Elderberry'))
print("Initial:")
inspect(table)

remove(table, 'Apricot')
print("After removing 'Apricot':")
inspect(table)

remove(table, 'Banana')
print("After removing 'Banana':")
inspect(table)